In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import cv2

import keras
import tensorflow
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as keras
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras_unet_collection import models, base, utils,losses
from tensorflow.keras.models import load_model
from keras_unet_collection.transformer_layers import patch_extract,patch_embedding,SwinTransformerBlock,patch_merging,patch_expanding

import glob
from skimage import morphology
from skimage.measure import label
from PIL import Image
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split,KFold
import time
import datetime


def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = keras.round(keras.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = keras.round(keras.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = keras.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = keras.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # keras.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + keras.epsilon())

    # return a single tensor value
    return recall

def load_test_data(test_npy_path):
    print('-'*30)
    print('load test images...')
    print('-'*30)

    imgs_test = cv2.imread(test_npy_path)
    imgs_test = imgs_test.astype('float32')
    imgs_test /= 255

    return imgs_test

def load_data(test_npy_path):
    imgs_test = np.array([load_test_data(test_npy_path)])
    print(imgs_test.shape)
    return imgs_test

In [2]:
def load_data2(npy_path):
    print('-'*30)
    print('load images...')
    print('-'*30)
    
    for i, path in enumerate(npy_path):
        print(path)
        train_npy_path = path.replace('_mask','')
        mask_npy_path = path
    
        imgs_tmp = np.load(train_npy_path)
        imgs_mask_tmp = np.load(mask_npy_path)
#         print(imgs_tmp.shape, imgs_mask_tmp.shape)
        
        if i==0:
            imgs = imgs_tmp
            imgs_mask = imgs_mask_tmp
            
        else:
            imgs = np.append(imgs, imgs_tmp, axis = 0)
            imgs_mask = np.append(imgs_mask, imgs_mask_tmp, axis = 0)
            
    print('-'*30)
    print('imgs : {} \nmasks : {}'.format(imgs.shape, imgs_mask.shape))    
    print('-'*30)
    
    imgs = imgs.astype('float32')
    imgs_mask = imgs_mask.astype('float32')
    print('img : ', imgs.max())
    print('mask : ',imgs_mask.max())

    print('-'*30)
    print('normalization start...')
    print('-'*30)
    
    imgs = imgs/255.0

    imgs_mask[imgs_mask<= 127] = 0
    imgs_mask[imgs_mask > 127] = 1

    print('img : ',imgs.max())
    print('mask : ',imgs_mask.max())

    return imgs, imgs_mask
from scipy import ndimage

def fill_hole(img):
    fill_img = ndimage.binary_fill_holes(img).astype(float)
    return fill_img

def fill_hole_cv(img):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(img, contours, -1, (255,255,255), -1)
    return img

In [3]:
img_path = '../../1. 데이터/4. 골반 분할 연구 데이터/'
all_patient = np.array([i.replace('_HE.png', '') for i in os.listdir(img_path) if '_HE.png' in i])

In [4]:
np.array([os.path.basename(i).replace('_HE.png', '') for i in glob.glob("res/*/*.png") if '_HE.png' in i])

array([], dtype=float64)

In [5]:
kf = KFold(n_splits=5, random_state=5, shuffle=True)
fold = []
for train_index, test_index in kf.split(all_patient):
    train_patient, test_patient = all_patient[train_index.astype(int)], all_patient[test_index.astype(int)]    
    
    testset = [img_path+'{}_HE.png'.format(p) for p in test_patient]
    trainset = [img_path+'{}_HE.png'.format(p) for p in train_patient]

    fold.append([trainset, testset])

In [6]:
for foldnum in range(5):
    testset = fold[foldnum][1]

   # 경로 설정 주의해서 보자

    model = load_model('6_result/exp_fold{}/model/last.h5'.format(foldnum),custom_objects={"dice": losses.dice, 'recall':recall})
    for test_path in testset:
        start = time.time()
        #test 이미지 로드 및 전처리
        imgs_test = load_data(test_path)
        print(imgs_test.shape)
        #predict
        imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)

        testset_name = test_path[4:]
        print(testset_name)
        name=testset_name
        ori_img_path = test_path
        
        #경로 설정 주의해서 보자 2
        pred_img_path = '6_result/exp_fold{}/pred_total/'.format(foldnum)
        pred_overimg_path = '6_result/exp_fold{}/pred_total_over/'.format(foldnum)
        if not os.path.isdir(pred_img_path):
            os.makedirs(pred_img_path)
        if not os.path.isdir(pred_overimg_path):
            os.makedirs(pred_overimg_path)
        os.makedirs(os.path.dirname(pred_img_path+name), exist_ok=True)
        os.makedirs(os.path.dirname(pred_overimg_path+name), exist_ok=True)
        
        #predict로 나온 결과 다시 0,255 threshold 및 fill hole과 morphology open을 이용한 후처리
        pred_list=imgs_mask_test[0]
        print("prediction image")
        imgs = pred_list
        img = imgs.astype('uint8')
        img[img <= 0.5] = 0
        img[img > 0.5] = 255
        print(img.shape)
        img = fill_hole_cv(img)
        k = np.ones((2,2), np.uint8)
        post_img = cv2.morphologyEx(img, cv2.MORPH_OPEN, k)
        cv2.imwrite(pred_img_path+name, post_img)
        
        #predict 결과와 골반 original 이미지의 overlay를 통한 결과 확인
        post_img = cv2.cvtColor(post_img, cv2.COLOR_GRAY2BGR)

        ori_img = load_img(ori_img_path)
        ori_img = img_to_array(ori_img)
        ori_img[:,:,0] = post_img[:,:,0]+ori_img[:,:,0]
        ori_img[:,:,0][ori_img[:,:,0]>=255]=255
        ori_img = array_to_img(ori_img)

        ori_img.save(pred_overimg_path+name)
        cp1 = time.time()
        sec = (cp1 - start) 
        result = datetime.timedelta(seconds=sec) 
        print(result)


2023-05-27 20:03:11.274281: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 20:03:11.829097: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22325 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:3b:00.0, compute capability: 8.6


------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)


2023-05-27 20:03:13.172824: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-05-27 20:03:14.234015: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


1/1 [==============================] - 4s 4s/step
./1. 데이터/4. 골반 분할 연구 데이터/539_HE.png
prediction image
(512, 512, 1)
0:00:04.402309
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 27ms/step
./1. 데이터/4. 골반 분할 연구 데이터/534_HE.png
prediction image
(512, 512, 1)
0:00:00.218485
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 24ms/step
./1. 데이터/4. 골반 분할 연구 데이터/529_HE.png
prediction image
(512, 512, 1)
0:00:00.223596
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 23ms/step
./1. 데이터/4. 골반 분할 연구 데이터/526_HE.png
prediction image
(512, 512, 1)
0:00:00.208078
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/

1/1 [==============================] - 0s 27ms/step
./1. 데이터/4. 골반 분할 연구 데이터/292_HE.png
prediction image
(512, 512, 1)
0:00:00.168882
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 23ms/step
./1. 데이터/4. 골반 분할 연구 데이터/289_HE.png
prediction image
(512, 512, 1)
0:00:00.183721
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/281_HE.png
prediction image
(512, 512, 1)
0:00:00.187845
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 25ms/step
./1. 데이터/4. 골반 분할 연구 데이터/280_HE.png
prediction image
(512, 512, 1)
0:00:00.202495
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)


...

1/1 [==============================] - 0s 22ms/step
./1. 데이터/4. 골반 분할 연구 데이터/64_HE.png
prediction image
(512, 512, 1)
0:00:00.178354
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 22ms/step
./1. 데이터/4. 골반 분할 연구 데이터/55_HE.png
prediction image
(512, 512, 1)
0:00:00.176826
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/52_HE.png
prediction image
(512, 512, 1)
0:00:00.187175
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/49_HE.png
prediction image
(512, 512, 1)
0:00:00.181431
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 

1/1 [==============================] - 0s 23ms/step
./1. 데이터/4. 골반 분할 연구 데이터/1160_HE.png
prediction image
(512, 512, 1)
0:00:00.173669
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/1154_HE.png
prediction image
(512, 512, 1)
0:00:00.191259
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 22ms/step
./1. 데이터/4. 골반 분할 연구 데이터/1146_HE.png
prediction image
(512, 512, 1)
0:00:00.183538
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/1145_HE.png
prediction image
(512, 512, 1)
0:00:00.209850
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512,

1/1 [==============================] - 0s 20ms/step
./1. 데이터/4. 골반 분할 연구 데이터/816_HE.png
prediction image
(512, 512, 1)
0:00:00.170619
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 22ms/step
./1. 데이터/4. 골반 분할 연구 데이터/814_HE.png
prediction image
(512, 512, 1)
0:00:00.163340
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/813_HE.png
prediction image
(512, 512, 1)
0:00:00.149010
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/812_HE.png
prediction image
(512, 512, 1)
0:00:00.161088
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)


1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/613_HE.png
prediction image
(512, 512, 1)
0:00:00.169633
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 23ms/step
./1. 데이터/4. 골반 분할 연구 데이터/606_HE.png
prediction image
(512, 512, 1)
0:00:00.194029
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 22ms/step
./1. 데이터/4. 골반 분할 연구 데이터/605_HE.png
prediction image
(512, 512, 1)
0:00:00.185011
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/593_HE.png
prediction image
(512, 512, 1)
0:00:00.174277
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)


1/1 [==============================] - 0s 19ms/step
./1. 데이터/4. 골반 분할 연구 데이터/650_HE.png
prediction image
(512, 512, 1)
0:00:00.137282
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/840_HE.png
prediction image
(512, 512, 1)
0:00:00.180940
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
./1. 데이터/4. 골반 분할 연구 데이터/357_HE.png
prediction image
(512, 512, 1)
0:00:00.184808
